# Use scikit-learn and custom library to predict temperature with `ibm-watson-machine-learning`

This notebook contains steps and code to train a Scikit-Learn model that uses a custom defined transformer and use it with Watson Machine Learning service. Once the model is trained, this notebook contains steps to persist the model and custom defined transformer to Watson Machine Learning Repository, deploy and score it using Watson Machine Learning python client.

In this notebook, we use GNFUV dataset that contains mobile sensor readings data about humidity and temperature from Unmanned Surface Vehicles in a test-bed in Athens, to train a Scikit-Learn model for predicting the temperature. 

Some familiarity with Python is helpful. This notebook uses Python-3.6, scikit-learn-0.20

## Learning goals

The learning goals of this notebook are:

- Train a model with custom defined transformer
- Persist the custom defined transformer and the model in Watson Machine Learning repository.
- Deploy the model using Watson Machine Learning Service
- Perform predictions using the deployed model

## Contents
1.	[Set up the environment](#setup)
2.	[Install python library containing custom transformer implementation](#install_lib)
3.  [Prepare training data](#load)
3.	[Train the scikit-learn model](#train)
4.	[Save the model and library to WML Repository](#persistence)
5.	[Deploy and score data in the IBM Cloud](#deploy)
6.	[Summary and next steps](#summary)


<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).


### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.


**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [3]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [4]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.


In [ ]:
!pip install -U ibm-watson-machine-learning

In [2]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [5]:
client.spaces.list(limit=10)

------------------------------------  --------------------------  ------------------------
ID                                    NAME                        CREATED
eb3c11ba-8600-4ab1-bbe1-125f774a923a  tests_sdk_space             2020-07-21T15:01:10.740Z
2d477c03-fab5-43da-8088-2764b4065a7b  Machine Learning-FVT-V2-LC  2020-07-20T14:18:48.093Z
------------------------------------  --------------------------  ------------------------


To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="install_lib"></a>

## 2. Install the library containing custom transformer

The library - `linalgnorm-0.1.zip` is a python distributable package that contains the implementation of a user defined Scikit-Learn transformer - `LNormalizer` . <br>
Any 3rd party libraries that are required for the custom transformer must be defined as the dependency for the corresponding library that contains implementation of the transformer. 


In this section, we download the library and install it in the current notebook environment. 

In [1]:
!wget https://github.com/IBM/watson-machine-learning-samples/raw/master/libraries/linalgnorm-0.1.zip --output-document=linalgnorm-0.1.zip

--2020-08-28 11:14:27--  https://github.com/IBM/watson-machine-learning-samples/raw/master/libraries/linalgnorm-0.1.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/libraries/linalgnorm-0.1.zip [following]
--2020-08-28 11:14:28--  https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/libraries/linalgnorm-0.1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2550 (2.5K) [application/zip]
Saving to: ‘linalgnorm-0.1.zip’

linalgnorm-0.1.zip  100%[===================>]   2.49K  --.-KB/s    in 0.02s   

2020-08-28 11:14:29 (99.7 KB/s) - ‘linalgnorm-0.1.zip’ saved [2550/2

Install the downloaded library using `pip` command

In [8]:
!pip install linalgnorm-0.1.zip

Processing ./linalgnorm-0.1.zip
  Created wheel for linalgnorm: filename=linalgnorm-0.1-cp36-none-any.whl size=1672 sha256=c3ffc0469321ac500802a58ad568bf95ca46a7c2ceb209cd72ac1f31ab4d37bc
  Stored in directory: /Users/Lukasz.Cmielowski@pl.ibm.com/Library/Caches/pip/wheels/1e/7c/18/ff38195f17f8a14833b1ff3a11491360bd0376da7f889c956e
Successfully built linalgnorm
  Found existing installation: linalgnorm 0.1
    Uninstalling linalgnorm-0.1:
      Successfully uninstalled linalgnorm-0.1


<a id="load"></a>

## 3. Download training dataset and prepare training data

Download the data from UCI repository - https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip

In [9]:
!rm -rf dataset
!mkdir dataset

In [10]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip --output-document=dataset/gnfuv_dataset.zip

--2020-08-27 14:50:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 501978 (490K) [application/x-httpd-php]
Saving to: ‘dataset/gnfuv_dataset.zip’

dataset/gnfuv_datas 100%[===================>] 490.21K   109KB/s    in 4.5s    

2020-08-27 14:50:22 (109 KB/s) - ‘dataset/gnfuv_dataset.zip’ saved [501978/501978]



In [16]:
!unzip dataset/gnfuv_dataset.zip -d dataset

Archive:  dataset/gnfuv_dataset.zip
  inflating: dataset/pi2/gnfuv-temp-exp1-55d487b85b-5g2xh_1.0.csv  
  inflating: dataset/pi3/gnfuv-temp-exp1-55d487b85b-2bl8b_1.0.csv  
  inflating: dataset/pi4/gnfuv-temp-exp1-55d487b85b-xcl97_1.0.csv  
  inflating: dataset/pi5/gnfuv-temp-exp1-55d487b85b-5ztk8_1.0.csv  
  inflating: dataset/README.pdf      


Create pandas datafame based on the downloaded dataset

In [17]:
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
from json import JSONDecodeError

In [19]:
home_dir = './dataset'
pi_dirs = os.listdir(home_dir)

data_list = []
base_time = None
columns = None

for pi_dir in pi_dirs:
    if 'pi' not in pi_dir:
        continue
    curr_dir = os.path.join(home_dir, pi_dir)
    data_file = os.path.join(curr_dir, os.listdir(curr_dir)[0])
    with open(data_file, 'r') as f:
        line = f.readline().strip().replace("'", '"')
        while line != '':
            try:
                input_json = json.loads(line)
                sensor_datetime = datetime.fromtimestamp(input_json['time'])
                if base_time is None:
                    base_time = datetime(sensor_datetime.year, sensor_datetime.month, sensor_datetime.day, 0, 0, 0, 0)
                input_json['time'] = (sensor_datetime - base_time).seconds
                data_list.append(list(input_json.values()))
                if columns is None:
                    columns = list(input_json.keys())
            except JSONDecodeError as je:
                pass
            line = f.readline().strip().replace("'", '"')

data_df = pd.DataFrame(data_list, columns=columns)

In [20]:
data_df.head()

,device,humidity,temperature,experiment,time
0,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69557
1,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69571
2,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69577
3,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69583
4,gnfuv-temp-exp1-55d487b85b-5g2xh,22.0,40.0,1.0,69589


Create training and test datasets from the downloaded GNFUV-USV dataset.

In [21]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Y = data_df['temperature']
X = data_df.drop('temperature', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=143)

<a id="train"></a>

## 4. Train a model

In this section, you will use the custom transformer as a stage in the Scikit-Learn `Pipeline` and train a model.

#### Import the custom transformer 
Here, import the custom transformer that has been defined in `linalgnorm-0.1.zip` and create an instance of it that will inturn be used as stage in `sklearn.Pipeline`

In [22]:
from linalg_norm.sklearn_transformers import LNormalizer

In [23]:
lnorm_transf = LNormalizer()

Import other objects required to train a model

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

Now, you can create a `Pipeline` with user defined transformer as one of the stages and train the model

In [25]:
skl_pipeline = Pipeline(steps=[('normalizer', lnorm_transf), ('regression_estimator', LinearRegression())])
skl_pipeline.fit(X_train.loc[:, ['time', 'humidity']].values, y_train)

Pipeline(memory=None,
     steps=[('normalizer', LNormalizer(norm_ord=2)), ('regression_estimator', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

In [26]:
y_pred = skl_pipeline.predict(X_test.loc[:, ['time', 'humidity']].values)
rmse = np.mean((np.round(y_pred) - y_test.values)**2)**0.5
print('RMSE: {}'.format(rmse))

RMSE: 2.213758431322581


<a id="persistence"></a>

## 5. Persist the model and custom library

In this section, using `ibm-watson_machine_learning` SDK, you will ...
- save the library `linalgnorm-0.1.zip` in WML Repository by creating a package extension resource
- create a Software Specification resource and bind the package resource to it. This Software Specification resource will be used to configure the online deployment runtime environment for a model 
- bind Software Specification resource to the model and save the model to WML Repository

### Create package extension

Define the meta data required to create package extension resource. <br>

The value for `file_path` in `client.package_extensions.LibraryMetaNames.store()` contains the library file name that must be uploaded to the WML.

**Note:** You can also use conda environment configuration file `yaml` as package extension input. In such case set the `TYPE` to `conda_yml` and `file_path` to yaml file.
```
client.package_extensions.ConfigurationMetaNames.TYPE = "conda_yml"
```

In [27]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "K_Linag_norm_skl",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Pkg extension for custom lib",
    client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}

pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="linalgnorm-0.1.zip")
pkg_extn_uid = client.package_extensions.get_uid(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)


Creating package extensions
SUCCESS


Display the details of the package extension resource that was created in the above cell.

In [ ]:
details = client.package_extensions.get_details(pkg_extn_uid)

### Create software specification and add custom library

Define the meta data required to create software spec resource and bind the package. This software spec resource will be used to configure the online deployment runtime environment for a model.

In [32]:
client.software_specifications.ConfigurationMetaNames.show()

---------------------------  ----  --------  --------------------------------
META_PROP NAME               TYPE  REQUIRED  SCHEMA
NAME                         str   Y
DESCRIPTION                  str   N
PACKAGE_EXTENSIONS           list  N
SOFTWARE_CONFIGURATION       dict  N         {'platform(required)': 'string'}
BASE_SOFTWARE_SPECIFICATION  dict  Y
---------------------------  ----  --------  --------------------------------


#### List base software specifications

In [33]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
tensorflow_2.1-py3.6           160fcf8b-0f8e-4043-ba83-e3bf021ea4fa  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
tensorflow_1.15-py3.6          2b73a275-7cbf-420b-a912-eae7f436e0bc  base
pytorch_1.2-py3.6              2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                2e51f700-bca0-4b0d-88dc-5c6791338875  base
pytorch-onnx_1.3-py3.6         2fcc03e

#### Select base software specification to extend

In [ ]:
base_sw_spec_uid = client.software_specifications.get_uid_by_name("scikit-learn_0.20-py3.6")

#### Define new software specification based on base one and custom library

In [35]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_uid = client.software_specifications.get_uid(sw_spec_details)


client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)

### Save the model

Define the metadata to save the trained model to WML Repository along with the information about the software spec resource required for the model. 

The `client.repository.ModelMetaNames.SOFTWARE_SPEC_UID` metadata property is used to specify the GUID of the software spec resource that needs to be associated with the model.

In [39]:
model_props = {
    client.repository.ModelMetaNames.NAME: "Temp prediction model with custom lib",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.20',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
    
}

Save the model to the WML Repository and display its saved metadata. 

In [40]:
published_model = client.repository.store_model(model=skl_pipeline, meta_props=model_props)

In [41]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "ab830ed6-e2fe-4c0f-9561-7a73df02faf6",
      "name": "linalgnorm-0.1"
    },
    "type": "scikit-learn_0.20"
  },
  "metadata": {
    "created_at": "2020-08-27T14:23:49.593Z",
    "id": "50765ddd-c01f-47ad-9863-da24a305b787",
    "modified_at": "2020-08-27T14:23:54.378Z",
    "name": "Temp prediction model with custom lib",
    "owner": "IBMid-270002BE4G",
    "space_id": "2d477c03-fab5-43da-8088-2764b4065a7b"
  }
}


<a id="deploy"></a>

## 6 Deploy and Score

In this section, you will deploy the saved model that uses the custom transformer and perform predictions. You will use WML client to perform these tasks.

### Deploy the model

In [43]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of custom lib model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)




#######################################################################################

Synchronous deployment creation for uid: '50765ddd-c01f-47ad-9863-da24a305b787' started

#######################################################################################


initializing..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='95a92e24-1bb9-4796-b27b-ea7cb77f2cb6'
------------------------------------------------------------------------------------------------




### Predict using the deployed model

**Note**: Here we use deployment `uid` saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.

In [46]:
deployment_uid = client.deployments.get_uid(created_deployment)

Now you can print an online scoring endpoint. 

In [47]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://wml-fvt.ml.test.cloud.ibm.com/ml/v4/deployments/95a92e24-1bb9-4796-b27b-ea7cb77f2cb6/predictions


Prepare the payload for prediction. The payload contains the input records for which predictions has to be performed.

In [48]:
scoring_payload = {
    "input_data": [{
        'fields': ["time", "humidity"],
        'values': [[79863, 47]]}]
}

Execute the method to perform online predictions and display the prediction results

In [49]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [50]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          14.629242312262988
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 6. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>

## 7. Summary

You successfully completed this notebook! 
 
You learned how to use a scikit-learn model with custom transformer in Watson Machine Learning service to deploy and score.

Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

## Author

**Krishnamurthy Arthanarisamy**, is a senior technical lead in IBM Watson Machine Learning team. Krishna works on developing cloud services that caters to different stages of machine learning and deep learning modeling life cycle.

**Lukasz Cmielowski**, PhD, is a Software Architect and Data Scientist at IBM.

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.